# COGS 108 - Final Project

## Permissions

Place an X in the appropriate bracket below to specify if you would like your group's project to be made available to the public. (Note that PIDs will be scraped from the public submission, but student names will be included.)

- [X] YES - make available
- [ ] NO - keep private

## Overview

## Names

- Gadi Rosen
- Rujvi Mehendre
- Henry Lozada
- Joshua Paz

## Group Members IDs

- A16105613
- A14553468
- A15127559
- A13300845

## Research Question

"_How has the outbreak of COVID-19 affected food security in communities in the greater San Diego area?_"

## Background & Prior Work

#### Why is this project of interest to your group?
 The pandemic has claimed many lives, but its impact on the US goes beyond that. People of limited financial means have been disproportionately affected by the stay-at-home orders issued. In light of this, we believe it’s important to consider how the pandemic, and the stay-at-home orders issued, affect people’s livelihoods and access to food.

#### What background information led to your hypothesis?
 Certain communities in the greater San Diego area had suffered from food insecurity prior to the advent of this pandemic1. According to an article on Fortune magazine2, since the beginning of the pandemic, unemployment has skyrocketed from 4% to nearly 18%. Another article by Fortune3, published 04/09/2020, states that “The total weekly claims fell close to 1.4 million from last week's 6.6 million initial unemployment claims.” Furthermore, according to an article published on yahoo! finance4, “Economists had been expecting the report to show the ranks of jobless Americans increasing by 5.5 million.” Meanwhile, information made available by the USDA5 indicates that food production has been decreasing as a result of the pandemic. Consequently, food banks are reporting an unprecedented increase in demand6, and the possibility of the US facing a food crisis is real.

#### Why is this important?
 We believe that this pandemic’s impact on the US goes beyond the lives it has claimed. The advent of the pandemic has resulted in stay-at-home orders which have increased the number of jobless Americans, limiting their financial resources and thus their ability to secure basic necessities, such as food. An analysis of the pandemic’s effect on the public’s access to food could result in a better understanding of the area’s food supply chain, which could be used by decision-makers and other relevant stakeholders to alleviate current shortages, and perhaps to prevent other shortages from occurring in the future.

#### What has already been done on this topic? What is already known?
 COVID-19 is a respiratory virus that is highly contagious and has infected well over 210 countries around the world, resulting in an on-going global pandemic. In order to curtail the spread of the disease, many governments have issued strict stay-at-home directives. These directives have resulted in the closure of businesses and increased unemployment.

 There have been studies and visualizations done in many countries worldwide highlighting spots where food insecurity is prevalent. These studies demonstrate that COVID-19 has only aggravated food insecurity, especially in countries where food insecurity was already prevalent.

#### References:
1. Food deserts and access to fresh food in low-income San Diego / by Emily Theresa Puhl
2. 22 million have lost their jobs over the past month—real unemployment rate likely nearing 18%
3. Real unemployment in the United States has likely hit 14.7%, the highest level since 1940
4. Jobless claims: Another 5.245 million Americans file for unemployment benefits
5. USDA - Quick Stats
6. A perfect storm: US facing hunger crisis as demand for food banks soars
7. COVID-19
#### Additional sources:
1. San Diego Hunger Coalition
2. UCSD Community Health
3. Foods Typically Purchased by Supplemental Nutrition Assistance Program (SNAP) Households
4. Food Security Information Network
5. Feeding San Diego
6. Neighborhood distribution Program
7. Food assistance during COVID-19 Pandemic
8. Greater Pittsburgh Community Foodbank - Our response to COVID-19
9. USDA - Coronavirus Disease
10. Nutritional Goals for Age-Sex Groups Based on Dietary Reference Intakes and Dietary Guidelines Recommendations

### Hypothesis:
The outbreak of COVID-19 has negatively impacted individuals’ access to food, disproportionately affecting the financially insecure and communities of color. These groups were already limited in their ability to provide for themselves and their families, and the pandemic has only made it more difficult for them to do so.

### Data:
1. San Diego Foodbank - Lbs of food distributed
2. SANDAG/SanGIS regional GIS data
2. SANDAG 2018 population estimates by Zip

### Setup

In [1]:
# Imports
%matplotlib inline
import fiona
import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (17, 5)
plt.rcParams.update({'font.size': 16})
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns

import shapely.geometry as shp

import sklearn.neighbors as skn
import sklearn.metrics as skm

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 10

#improve resolution
#comment this line if erroring on your machine/screen
%config InlineBackend.figure_format ='retina'

In [ ]:
filepath = "Census_Data/Census_ZIP.shp"
tempmap = gpd.read_file(filepath)
pop = pd.read_excel("Household Population ZIP.xlsx")
sdmap= pd.merge(tempmap, pop, on='ZIP', how='left')

In [ ]:
#Population by Zip code
fig, ax = plt.subplots(1, 1, figsize=(17, 7))
divider = make_axes_locatable(ax)
sdmap.plot(column='POPULATION', ax=ax, cmap='GnBu', legend=True);

In [ ]:
addresses = gpd.read_file('ADDRESS_APN.shp')

In [ ]:
addresses.head(5)
#addresses[addresses['ADDRFRAC'] != "None"]
#Check to see if 14925 Great Southern Overland Stage in addresses
#Is "Glen", "Broadway" and ADDRNAME?
#How to handle highways? Ex: 4141 Pacific Highway
#addresses['ADDRSFX'].unique()

In [ ]:
#Read in all foodbank data
#2019
Jan2019 = pd.read_excel("San_Diego_Foodbank_Data/Jan 2019.xlsx")
Feb2019 = pd.read_excel("San_Diego_Foodbank_Data/Feb 2019.xlsx")
Mar2019 = pd.read_excel("San_Diego_Foodbank_Data/Mar 2019.xlsx")
Apr2019 = pd.read_excel("San_Diego_Foodbank_Data/Apr 2019.xlsx")
#2020
Jan2020 = pd.read_excel("San_Diego_Foodbank_Data/Jan 2020.xlsx")
Feb2020 = pd.read_excel("San_Diego_Foodbank_Data/Feb 2020.xlsx")
Mar2020 = pd.read_excel("San_Diego_Foodbank_Data/Mar 2020.xlsx")
Apr2020 = pd.read_excel("San_Diego_Foodbank_Data/Apr 2020.xlsx")

#Note: Number of foodbanks sampled varies from year to year and month to month.
#----------------------------------------------------------------------------------
    # Jan 2019: 452
    # Feb 2019: 387
    # Mar 2019: 431
    # Apr 2019: 418
    #--------------
    # Jan 2020: 486
    # Feb 2020: 331
    # Mar 2020: 481
    # Apr 2020: 454
#----------------------------------------------------------------------------------
# Suggestion: Consider only foodbanks that consistently appear throughout all dfs.
# Solution: Merge all of the dfs by inner?
col = pd.merge(Jan2019, Feb2019, on='STREET ADDRESS', how='inner')
col = pd.merge(col, Mar2019, on='STREET ADDRESS', how='inner')
col = pd.merge(col, Apr2019, on='STREET ADDRESS', how='inner')
col = pd.merge(col, Jan2020, on='STREET ADDRESS', how='inner')
col = pd.merge(col, Feb2020, on='STREET ADDRESS', how='inner')
col = pd.merge(col, Mar2020, on='STREET ADDRESS', how='inner')
col = pd.merge(col, Apr2020, on='STREET ADDRESS', how='inner')
col.drop_duplicates(subset="STREET ADDRESS", inplace=True)

In [ ]:
# If done correctly, there are only 193 foodbanks that appear in all records. We will work exclusively with these.
col

In [ ]:
# Write function to split STREET ADDRESS into:
#     1. ADDRNMBR
#     2. ADDRFRAC
#     3. ADDRNAME
#     4. ADDRUNIT
""" 
1. Avenue X
2. Drive X
3. Street X
4. Ave X
5. Lane X
6. St. X
7. Way X
8. Dr. X
9. Ave. X
10. St X
11. Road X
12. Dr X
13. Blvd X
14. Rd X
15. Boulevard X
16. Glen?
17. Trail
18. Circle
19. Terrace
20. Parkway
21. Place
22. Park
23. Court
24. Route"""


def standardize_address(string):
    try:
        string = string.upper()
        string = string.split(" ", 1)
        if "-" in string[0]:
            string[0].split("-", 1)
            addrnumber = string[0].split("-")[0]
            addrunit = string[0].split("-")[1]
        else:
            addrnumber = float(string[0])
        addrsfx = ""
        addrsuite = ""
        addrname = ""
        print(string)
        string = string[1]
        print("after strip" + string)
        if "," in string:
            string = string.rsplit(",", 1)
            print(string)
            addrsuite = string[1].strip()
            string = string[0]
        AveSuffixes = ['Avenue', 'Ave', 'Ave.']
        DriveSuffixes = ['Drive', 'Dr', 'Dr.']
        StreetSuffixes = ['Street', 'St.', 'St']
        RoadSuffixes = ['Road', 'Rd']
        BoulevardSuffixes = ['Boulevard', 'Blvd']
        print(string)
        for e in AveSuffixes:
            if e.upper() in string:
                addrsfx = 'AVE'
                string = string.split(e.upper(), 1)

        for e in RoadSuffixes:
            if e.upper() in string and "BROADWAY" not in string:
                addrsfx = 'RD'
                string = string.split(e.upper(), 1)

        for e in StreetSuffixes:
            if e.upper() in string:
                addrsfx = 'ST'
                string = string.split(e.upper(), 1)

        for e in DriveSuffixes:
            if e.upper() in string:
                addrsfx = 'DR'
                string = string.split(e.upper(), 1)

        for e in BoulevardSuffixes:
            if e.upper() in string:
                addrsfx = 'BLVD'
                string = string.split(e.upper(), 1)
        print(string)
        if "LANE" in string:
            addrsfx = 'LN'
            string = string.split("LANE", 1)

        elif " WAY" in string and "BROADWAY" not in string:
            addrsfx = 'WAY'
            string = string.split("WAY", 1)
            print("inside way" + string)
        elif "BROADWAY" in string:
            addrname = "BROADWAY"
            print("inside broadway" + string)
        elif "GLEN" in string:
            addrsfx = 'GLEN'
            string = string.split("GLEN", 1)

        elif "PARKWAY" in string:
            addrsfx = "PKWY"
            string = string.split("PARKWAY", 1)

        elif "TRAIL" in string:
            addrsfx = "TRL"
            string = string.split("TRAIL", 1)

        elif "CIRCLE" in string:
            addrsfx = "CIR"
            string = string.split("CIRCLE", 1)

        elif "TERRACE" in string:
            addrsfx = "TER"
            string = string.split("TERRACE", 1)

        elif "PLACE" in string:
            addrsfx = "PL"
            string = string.split("PLACE", 1)

        elif "PARK" in string:
            addrsfx = "PARK"
            string = string.split("PARK", 1)

        elif "COURT" in string:
            addrsfx = "CT"
            string = string.split("COURT", 1)

        elif "ROUTE" in string:
            addrsfx = "RTE" # Try RT if this doesn't work
            string = string.split("ROUTE", 1)
        print(string)
        if(addrsfx!=""):
            addrname = string[0].strip()

        return addrnumber, addrname, addrsfx, addrsuite
    
    except:
        print("no good")



    # check rows, cols, check weird cases
    # unique rows and columns with .unique() function
    # printing every column in terms of a list

In [ ]:
# initialize list of lists 
  
# Create the pandas DataFrame 
#data = pd.DataFrame(data, columns = ['number', 'name', 'suffix', 'suite', 'unit'])
#temp = col.iloc[0,2]
#temp
standardize_address('542 Grape Vie Road, ste 200')

In [ ]:
incomefile = pd.read_excel("income_sd.xlsx")
incomefile = incomefile.loc[incomefile['YEAR'] == 2018]
incomefile = incomefile.reset_index()
incomefile.head(10)
len(incomefile['ZIP'].unique())

In [ ]:
incomefile['INCOME GROUP'] = incomefile['INCOME GROUP'].astype(str)
dataTypeSeries =  incomefile.dtypes
 
print('Data type of each column of Dataframe :')
#print(dataTypeSeries)
#incomefile

incomefile

In [ ]:
incomefile['INCOME GROUP'].replace({"Less than $15,000": 10000}, inplace=True)
incomefile['INCOME GROUP'].replace({"$15,000 to $29,999": 23000}, inplace=True)
incomefile['INCOME GROUP'].replace({"$30,000 to $44,999": 37500}, inplace=True)
incomefile['INCOME GROUP'].replace({"$45,000 to $59,999": 52500}, inplace=True)
incomefile['INCOME GROUP'].replace({"$60,000 to $74,999": 67500}, inplace=True)
incomefile['INCOME GROUP'].replace({"$75,000 to $99,999": 87500}, inplace=True)
incomefile['INCOME GROUP'].replace({"$100,000 to $124,999": 112500}, inplace=True)
incomefile['INCOME GROUP'].replace({"$125,000 to $149,999": 137500}, inplace=True)
incomefile['INCOME GROUP'].replace({"$150,000 to $199,999": 175000}, inplace=True)
incomefile['INCOME GROUP'].replace({"$200,000 or more": 220000}, inplace=True)
incomefile

In [ ]:
#grouped = incomefile.groupby('ZIP')
df = incomefile.groupby('ZIP').apply(lambda x: x['INCOME GROUP']*x['HOUSEHOLDS'])
#print(incomefile)
df2 = incomefile.groupby('ZIP')['HOUSEHOLDS'].sum()
#print(df2)
df = df.groupby('ZIP').sum()

#df = df.groupby('ZIP')
#df = df.apply(lambda x: x['INCOME GROUP']/x['HOUSEHOLDS'])

#df2 = df.apply(lambda x: x['INCOME GROUP']/x['HOUSEHOLDS'])
df.dtypes
df = df.reset_index() 
#print(df)
df.columns = ['ZIP', 'TOTAL INCOME']
df2.colums = ['ZIP', 'HOUSEHOLDS']
newincome = df.merge(df2, on = 'ZIP', how = 'inner')
print(newincome)
income = newincome.groupby('ZIP')
income = income.apply(lambda x: x['TOTAL INCOME']/x['HOUSEHOLDS'])
income = income.reset_index()
income.columns = ['ZIP', 'IND', 'AVERAGE INCOME']
val = income[income['AVERAGE INCOME'] == income['AVERAGE INCOME'].max()]
less = income[income['AVERAGE INCOME'] == income['AVERAGE INCOME'].min()]
print(val)
print(less)
income

In [ ]:
#filepath1 = "Census_Data/Census_ZIP.shp"
#tempmap1 = gpd.read_file(filepath1)
#pop = pd.read_excel("Household Population ZIP.xlsx")
#sdmap1= pd.merge(tempmap1, income, on='ZIP', how='left')
duplicateRowsDF = income[income.duplicated()]
income.dropna()

In [ ]:
filepath1 = "CENSUS-Data/Census_ZIP.shp"
tempmap1 = gpd.read_file(filepath1)


fig1, ax1 = plt.subplots(1, 1, figsize=(17, 7))
divider1 = make_axes_locatable(ax1)
sdmap1= pd.merge(tempmap1, income, on='ZIP', how='left')
sdmap1 = sdmap1.dropna()
sdmap1.plot(column='AVERAGE INCOME', ax=ax1, cmap='GnBu', legend=True);

In [13]:
eth = pd.read_excel("ethnicity.xlsx")
eth

,ZIP,YEAR,ETHNICITY,POPULATION
0,91901,2010,American Indian,266
1,91901,2010,Asian,328
2,91901,2010,Black,184
3,91901,2010,Hispanic,2690
4,91901,2010,Other,10
...,...,...,...,...
8131,92672,2018,Hispanic,1307
8132,92672,2018,Other,11
8133,92672,2018,Pacific Islander,22
8134,92672,2018,Two or More,161


In [14]:
eth['ETHNICITY'].unique()
eth = eth[eth['YEAR'] == 2018]

In [17]:
def prop(df):
    props = []
    for i in range(len(df)):
        zipc = df.iloc[i]['ZIP']
        if zipc in pop.index:
            props.append(df.iloc[i]['POPULATION']/pop[zipc])
        else:
            continue
    df['PROPORTION'] = props
    return

In [18]:
ethAI = eth[eth['ETHNICITY'] == 'American Indian']
ethA = eth[eth['ETHNICITY'] == 'Asian']
ethB = eth[eth['ETHNICITY'] == 'Black']
ethH = eth[eth['ETHNICITY'] == 'Hispanic']
ethO = eth[eth['ETHNICITY'] == 'Other']
ethPI = eth[eth['ETHNICITY'] == 'Pacific Islander']
ethTM = eth[eth['ETHNICITY'] == 'Two or More']
ethW = eth[eth['ETHNICITY'] == 'White']
prop(ethAI)
prop(ethA)
prop(ethB)
prop(ethH)
prop(ethO)
prop(ethPI)
prop(ethTM)
prop(ethW)

NameError: name 'pop' is not defined